In [8]:
import pandas as pd

In [9]:
df = pd.read_excel('test1.xlsx')
df

,seq,ship,loc,arr_date,mot
0,1,123456,egdam,2023-01-02,tr
1,2,123456,egpsd,2023-01-05,ve
2,3,123456,egsok,2023-01-08,tr
3,1,565758,egcai,2023-01-05,ve
4,2,565758,egdam,2023-02-05,ve
5,3,565758,egsok,2023-03-10,tr
6,4,565758,egcai,2023-04-10,tm


In [10]:
# Group by 'ship' and sort by 'seq' within each group
grouped = df.sort_values(by=['ship', 'seq'])
grouped

,seq,ship,loc,arr_date,mot
0,1,123456,egdam,2023-01-02,tr
1,2,123456,egpsd,2023-01-05,ve
2,3,123456,egsok,2023-01-08,tr
3,1,565758,egcai,2023-01-05,ve
4,2,565758,egdam,2023-02-05,ve
5,3,565758,egsok,2023-03-10,tr
6,4,565758,egcai,2023-04-10,tm


In [11]:
# Create a shifted DataFrame with an extra column to help in merging
shifted_df = grouped.copy()
shifted_df['seq_shifted'] = shifted_df['seq'] + 1
shifted_df

,seq,ship,loc,arr_date,mot,seq_shifted
0,1,123456,egdam,2023-01-02,tr,2
1,2,123456,egpsd,2023-01-05,ve,3
2,3,123456,egsok,2023-01-08,tr,4
3,1,565758,egcai,2023-01-05,ve,2
4,2,565758,egdam,2023-02-05,ve,3
5,3,565758,egsok,2023-03-10,tr,4
6,4,565758,egcai,2023-04-10,tm,5


In [12]:
# Merge the original and shifted dataframes on 'ship' and the new 'seq_shifted' column
merged_df = pd.merge(grouped, shifted_df, left_on=['ship', 'seq'], right_on=['ship', 'seq_shifted'], how='left')
merged_df

,seq_x,ship,loc_x,arr_date_x,mot_x,seq_y,loc_y,arr_date_y,mot_y,seq_shifted
0,1,123456,egdam,2023-01-02,tr,NaN,NaN,NaT,NaN,NaN
1,2,123456,egpsd,2023-01-05,ve,1.0,egdam,2023-01-02,tr,2.0
2,3,123456,egsok,2023-01-08,tr,2.0,egpsd,2023-01-05,ve,3.0
3,1,565758,egcai,2023-01-05,ve,NaN,NaN,NaT,NaN,NaN
4,2,565758,egdam,2023-02-05,ve,1.0,egcai,2023-01-05,ve,2.0
5,3,565758,egsok,2023-03-10,tr,2.0,egdam,2023-02-05,ve,3.0
6,4,565758,egcai,2023-04-10,tm,3.0,egsok,2023-03-10,tr,4.0


In [13]:
# Select and rename columns to match test2 structure
result_df = merged_df[['ship', 'loc_x', 'loc_y', 'mot_y', 'arr_date_x', 'arr_date_y']]
result_df.columns = ['ship', 'loc-1', 'loc-2', 'mot', 'arr_date_loc-1', 'last_date_loc-2']

In [14]:
# Drop rows with NaN values (which appear due to the shift)
result_df = result_df.dropna()

In [15]:
# Display the result
result_df.head()

,ship,loc-1,loc-2,mot,arr_date_loc-1,last_date_loc-2
1,123456,egpsd,egdam,tr,2023-01-05,2023-01-02
2,123456,egsok,egpsd,ve,2023-01-08,2023-01-05
4,565758,egdam,egcai,ve,2023-02-05,2023-01-05
5,565758,egsok,egdam,ve,2023-03-10,2023-02-05
6,565758,egcai,egsok,tr,2023-04-10,2023-03-10


In [17]:
# Revised code with a new sequential identifier for each row
test1_df = pd.read_excel('test1.xlsx')
# Sort by ship and seq
test1_df_sorted = test1_df.sort_values(by=['ship', 'seq'])

# Add a sequential identifier to assist with the merge
test1_df_sorted['row_id'] = range(len(test1_df_sorted))

# Shift the dataframe to get the next location, mot, and arr_date
test1_df_shifted = test1_df_sorted.copy()
test1_df_shifted['row_id'] -= 1  # shift the row_id to align with the previous row

# Merge the original and shifted dataframe based on row_id
merged_df = pd.merge(test1_df_sorted, test1_df_shifted, left_on='row_id', right_on='row_id', suffixes=('', '_next'))

# Filter out rows where ship number doesn't match in the combined rows
result_df = merged_df[merged_df['ship'] == merged_df['ship_next']]

# Select and rename columns to match test2 structure
result_df = result_df[['ship', 'loc', 'loc_next', 'mot_next', 'arr_date', 'arr_date_next']]
result_df.columns = ['ship', 'loc-1', 'loc-2', 'mot', 'arr_date_loc-1', 'last_date_loc-2']

# Display the result
result_df.head()


,ship,loc-1,loc-2,mot,arr_date_loc-1,last_date_loc-2
0,123456,egdam,egpsd,ve,2023-01-02,2023-01-05
1,123456,egpsd,egsok,tr,2023-01-05,2023-01-08
3,565758,egcai,egdam,ve,2023-01-05,2023-02-05
4,565758,egdam,egsok,tr,2023-02-05,2023-03-10
5,565758,egsok,egcai,tm,2023-03-10,2023-04-10
